# Interactive use of PAHFIT and cube fitting

Main goal: extract 1D spectrum from cube, then run PAHFIT on it so see how well it works.

In [ ]:
%matplotlib inline
from specutils import Spectrum1D
from jdaviz import Cubeviz
from matplotlib import pyplot as plt
from astropy import units as u

def plot_spectrum1d(spec1d, ax=None, **kwargs):
    if ax is None:
        f, (ax1) = plt.subplots(1, 1, figsize=(15, 5))
    else:
        ax1 = ax
    ax1.set_xlabel("Observed Wavelength (microns)")  
    ax1.set_ylabel(f"flux ({spec1d.flux.unit})")
    ax1.plot(spec1d.spectral_axis.to(u.micron).value, spec1d.flux.value, **kwargs)
    ax1.legend()
    return ax1

In [ ]:
# MRS cube
filename = "stage3-boris/BandCube_ch1-long_s3d.fits"

In [ ]:
# show the cube
cubeviz = Cubeviz()
cubeviz.app.load_data(filename)
cubeviz.app

## Extract 1D spectrum

Spectrum1D objects can be extracted from CubeViz

If a region is selected, and a collapsed spectrum was calculated, the result can be extracted from the spec-viewer. See https://jdaviz.readthedocs.io/en/latest/notebook/export_data.html?highlight=extract#export-spectral-regions

Or, you can work with the coordinates of the selected region (which can be found as shown below), and use other tools on the data directly.

In [ ]:
regions = cubeviz.app.get_subsets_from_viewer('flux-viewer')
region1_exists = 'region 1' in regions
if region1_exists:
    x0, y0 = region1.center.x, region1.center.y
    r_pix = region1.radius

For the PAHFIT demonstration below, we load in the cube using Spectrum1D (works out of the box with JWST), and take a random spaxel.

In [ ]:
spec1d_cube = Spectrum1D.read(filename)
ny, nx, nw = spec1d_cube.shape
# take central pixel
spectrum = spec1d_cube[ny // 2, nx // 2]
plot_spectrum1d(spectrum)

## Run PAHFIT on Spectrum1D object

By importing PAHFIT, we can use one of the utility functions to load the science pack and construct the corresponding model. 

The goal of this part is to show that PAHFIT can be used interactively, starting from a Spectrum1D object. 

Doing this allows us to quickly judge how good the chosen science pack is working. This is useful for PAHFIT development, e.g. exploring for which regions of the PDR the science pack is working.

In [ ]:
from pahfit.helpers import initialize_trimmed_model, fit_spectrum
from pahfit.base import PAHFITBase

def fit(spec1d):
    # convert Spectrum1D object to PAHFIT data format
    obsdata = {"x": spec1d.spectral_axis.to(u.micron), "y": spec1d.flux, "unc": spec1d.uncertainty.quantity}
    print(obsdata)
    
    # make the model
    model_base = initialize_trimmed_model('scipack_ExGal_SpitzerIRSSLLL.ipac', obsdata)
    
    # run the fit
    model_result = fit_spectrum(obsdata, model_base, maxiter=1000)
    print(model_result)
    
    # plot separate components
    fig, ax = plt.subplots(2, 1, figsize=(10,10))
    PAHFITBase.plot(ax, obsdata['x'], obsdata['y'], obsdata['unc'], model_result)

In [ ]:
fit(spectrum)

In [ ]:
plt.show()